In [1]:
!pip install ultralytics


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import zipfile

zip_path = "/dgxa_home/se22uari081/amazon_bin.v1i.yolov8.zip"
out_dir = "/dgxa_home/se22uari081/amazon_bin_dataset"

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(out_dir)

print("EXTRACTED to:", out_dir)


EXTRACTED to: /dgxa_home/se22uari081/amazon_bin_dataset


In [4]:
import os

for root,dirs,files in os.walk(out_dir):
    print(root)
    break


/dgxa_home/se22uari081/amazon_bin_dataset


In [6]:
import os, shutil, random

BASE = "/dgxa_home/se22uari081/amazon_bin_dataset"

train_img = f"{BASE}/train/images"
train_lbl = f"{BASE}/train/labels"

valid_img = f"{BASE}/valid/images"
valid_lbl = f"{BASE}/valid/labels"

os.makedirs(valid_img, exist_ok=True)
os.makedirs(valid_lbl, exist_ok=True)

# move 10% files to valid/
images = os.listdir(train_img)
random.shuffle(images)

val_size = int(len(images) * 0.1)
val_files = images[:val_size]

for f in val_files:
    shutil.move(os.path.join(train_img, f), valid_img)
    lbl = f.replace(".jpg",".txt")
    shutil.move(os.path.join(train_lbl, lbl), valid_lbl)

print("VALID set created.")


VALID set created.


In [5]:
import os, shutil, random

BASE = "/dgxa_home/se22uari081/amazon_bin_dataset"

os.makedirs(f"{BASE}/valid/images", exist_ok=True)
os.makedirs(f"{BASE}/valid/labels", exist_ok=True)

train_img = f"{BASE}/train/images"
train_lbl = f"{BASE}/train/labels"

images = os.listdir(train_img)
random.shuffle(images)

val_size = max(30, int(len(images) * 0.1))

for f in images[:val_size]:
    shutil.move(f"{train_img}/{f}", f"{BASE}/valid/images/{f}")
    shutil.move(f"{train_lbl}/{f.replace('.jpg','.txt')}", f"{BASE}/valid/labels/{f.replace('.jpg','.txt')}")


In [6]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

model.train(
    data="/dgxa_home/se22uari081/amazon_bin_dataset/data.yaml",
    epochs=50,
    imgsz=640,
    batch=8
)


Ultralytics 8.3.231 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB MIG 1g.5gb, 4864MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/dgxa_home/se22uari081/amazon_bin_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patien

/dgxa_home/se22uari081/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 8.1±4.5 MB/s, size: 25.8 KB)
val: Scanning /dgxa_home/se22uari081/amazon_bin_dataset/valid/labels... 63 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 63/63 1.2Kit/s 0.1s
val: New cache created: /dgxa_home/se22uari081/amazon_bin_dataset/valid/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 260, len(boxes) = 268. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


/dgxa_home/se22uari081/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Plotting labels to /dgxa_home/se22uari081/runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /dgxa_home/se22uari081/runs/detect/train4
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/50         2G      2.058      2.777      2.163         12        640: 100% ━━━━━━━━━━━━ 13/13 3.5it/s 3.7s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 4.5it/s 0.9s0.3s
                   all         63        268      0.146      0.284      0.107     0.0332

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       S

/dgxa_home/se22uari081/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      41/50      2.16G      1.354      1.529      1.655         10        640: 100% ━━━━━━━━━━━━ 13/13 3.1it/s 4.1s0.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 6.6it/s 0.6s0.2s
                   all         63        268      0.689       0.53      0.587      0.342

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      42/50      2.16G      1.325      1.519      1.671         10        640: 100% ━━━━━━━━━━━━ 13/13 6.2it/s 2.1s0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 6.5it/s 0.6s0.2s
                   all         63        268      0.653      0.534      0.585      0.337

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      43/50      2.07G      1.371      1.595      1.709          6        640: 100% ━━━

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f579ea4bfd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [7]:
from ultralytics import YOLO

model = YOLO("/dgxa_home/se22uari081/runs/detect/train4/weights/best.pt")


In [13]:
results = model("bin_data/images/00821.jpg", save=True)



image 1/1 /dgxa_home/se22uari081/bin_data/images/00821.jpg: 448x640 9 objects-b8uQs, 7.3ms
Speed: 1.5ms preprocess, 7.3ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)
Results saved to /dgxa_home/se22uari081/runs/detect/predict


In [14]:
results = model("bin_data/images/00821.jpg")[0]
results.show()



image 1/1 /dgxa_home/se22uari081/bin_data/images/00821.jpg: 448x640 9 objects-b8uQs, 6.8ms
Speed: 1.7ms preprocess, 6.8ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)


display-im6.q16: unable to open X server `localhost:31.0' @ error/display.c/DisplayImageCommand/412.


In [15]:
from matplotlib import pyplot as plt

plt.imshow(results.plot())
plt.axis('off')


(np.float64(-0.5), np.float64(368.5), np.float64(251.5), np.float64(-0.5))

In [17]:
results = model("bin_data/images/00821.jpg")[0]

counts = {}
for box in results.boxes:
    cls = model.names[int(box.cls)]
    counts[cls] = counts.get(cls, 0) + 1

counts



image 1/1 /dgxa_home/se22uari081/bin_data/images/00821.jpg: 448x640 9 objects-b8uQs, 6.9ms
Speed: 1.7ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)


{'objects-b8uQ': 9}

In [18]:
import random

imgs = os.listdir("bin_data/images")
sample = random.choice(imgs)

results = model(f"bin_data/images/{sample}", save=True)
print("saved:", sample)



image 1/1 /dgxa_home/se22uari081/bin_data/images/186568.jpg: 448x640 5 objects-b8uQs, 6.6ms
Speed: 1.5ms preprocess, 6.6ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)
Results saved to /dgxa_home/se22uari081/runs/detect/predict
saved: 186568.jpg
